In [15]:
# Matplotlib
import matplotlib.pyplot as plt
# Numpy
import numpy as np
# Torch
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

import os
import pandas as pd





In [23]:
# Load dataset manually using NumPy
file_path = "reno1.log.csv"  # Update with actual file path
# Load the dataset using numpy (semicolon separated)
df = pd.read_csv(file_path, delimiter=";")  # skip_header=1 if there’s a header row

# remove last column, some kind of spacing i
# Drop the last column using pandas
df = df.iloc[:, :-1]  # Select all rows, and all columns except the last one

df['loss_ratio'] = df['bytes_retrans'] / df['bytes_sent']
# Identify constant columns
constant_columns = [col for col in df.columns if len(df[col].unique()) == 1]

constant_columns.append("ssthresh")
print("colunms without any variation: ", constant_columns)
# Drop constant columns from the dataset
df = df.drop(columns=constant_columns)

seq_length = 15

#Function to create sequences\n",
def create_sequences(input, labels, seq_length):
    xs, ys = [], [],
    for i in range(len(input) - seq_length):
        xs.append(input[i : i + seq_length])  # Input sequence
        ys.append(labels[i + seq_length])   # Corresponding target
    return np.array(xs), np.array(ys)


X, y = create_sequences(df.values, df['loss_ratio'].values, seq_length)

trainX = torch.tensor(X,dtype=torch.float32)
trainY = torch.tensor(y[:, None], dtype=torch.float32)

print("Data tensor:", X.shape)
print("Loss Ratio tensor:", y.shape)

colunms without any variation:  ['wscale', 'mss', 'pmtu', 'rcvmss', 'advmss', 'rcv_space', 'rcv_ssthresh', 'ssthresh']
Data tensor: (3585, 15, 12)
Loss Ratio tensor: (3585,)


In [17]:
# Use GPU if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [19]:
class LSTM_pt(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM_pt, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        # LSTM cell
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers = self.num_layers, batch_first = True)
        
        # Linear layer for final prediction
        self.linear = torch.nn.Linear(hidden_size, output_size)

    def forward(self, inputs, cell_state=None, hidden_state=None):
        # Forward pass through the LSTM cell
        if hidden_state is None or cell_state is None:
            hidden_state = torch.zeros(1, inputs.size(0), 20).to(device)
            cell_state = torch.zeros(1, inputs.size(0), 20).to(device)
        hidden = (cell_state, hidden_state)
        output, new_memory = self.lstm(inputs, hidden)
        cell_state, hidden_state = new_memory
        output = self.linear(output[:, -1, :])  # Take only the last time step
        return output, cell_state, hidden_state, # Return correct order

In [20]:
input_size = 12
hidden_size = 20
num_layers = 1 # Can be changed to stack multiple LSTM layers!
output_size = 1

model = LSTM_pt(input_size, hidden_size, num_layers, output_size).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# np.random.seed(0)
# torch.manual_seed(0)

# t = np.linspace(0, 100, 1000)
# data = np.sin(t)

# def create_sequences(data, seq_length):
#     xs = []
#     ys = []
#     for i in range(len(data)-seq_length):
#         x = data[i:(i+seq_length)]
#         y = data[i+seq_length]
#         xs.append(x)
#         ys.append(y)
#     return np.array(xs), np.array(ys)

# seq_length = 10
# X, y = create_sequences(data, seq_length)

# trainX = torch.tensor(X[:, :, None], dtype=torch.float32)
# trainY = torch.tensor(y[:, None], dtype=torch.float32)
# print("trainX shape: ", trainX.shape)
# print("trainY shape: ", trainY.shape)

In [21]:
num_epochs = 100
h0, c0 = None, None

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    outputs, h0, c0 = model(trainX, h0, c0)

    loss = criterion(outputs, trainY)

    loss.backward()
    optimizer.step()

    h0 = h0.detach()
    c0 = c0.detach()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: nan
Epoch [20/100], Loss: nan
Epoch [30/100], Loss: nan
Epoch [40/100], Loss: nan
Epoch [50/100], Loss: nan
Epoch [60/100], Loss: nan
Epoch [70/100], Loss: nan
Epoch [80/100], Loss: nan
Epoch [90/100], Loss: nan
Epoch [100/100], Loss: nan
